In [2]:
include("astrobee_se3_script.jl")
using Plots

In [3]:
function solve_gusto(MAX_ITERATIONS_NB, X, U, Xp, Up, model, Delta0, omega0, omegamax, 
    epsilon, rho0, rho1, 
    beta_succ, beta_fail, gamma_fail, 
    convergence_threshold)
    
    Delta = Delta0
    omega = omega0
    
    B_success = false
    it = 0
    X_all, U_all = [], []
    push!(X_all, copy(X))
    push!(U_all, copy(U))
    while it<MAX_ITERATIONS_NB && 
            !(it!=0 && it!=1 && it!=2 && it!=3 && B_success && 
                convergence_metric(model,X,U,Xp,Up)<convergence_threshold) &&
            omega<omegamax


    #     omega, Delta = SCPproblem.omega, SCPproblem.Delta

        Xp = copy(X)
        Up = copy(U)

        model.f, model.A, model.B = compute_dynamics(model, Xp, Up)

        reset_problem(SCPproblem, model)
        set_parameters(SCPproblem, model, Xp, Up, omega, Delta)
        define_cost(SCPproblem, model)
        define_constraints(SCPproblem, model)

        JuMP.optimize!(SCPproblem.solver_model)    
        p0 = get_initial_constraint_dual_variable(SCPproblem, model)
        # get solution
        X_sol = JuMP.value.(SCPproblem.X)
        U_sol = JuMP.value.(SCPproblem.U)

        # try shooting
        #x_shooting, sol_newton = solve_shooting(SCPproblem, model, p0)
    #     x_shooting = solve_shooting_once(SCPproblem, model, p0)
        #push!(x_shooting_all, x_shooting)

        # -----------
        # GuSTO Logic
        if it > 3
            if is_in_trust_region(model, X_sol, U_sol, Xp, Up, Delta)
                rho = accuracy_ratio(SCPproblem, model, X_sol, U_sol, Xp, Up)

                if rho > rho1
                    Delta = beta_fail * Delta
                    omega     = omega
                    B_success = false

                else
                    X = copy(X_sol)
                    U = copy(U_sol)
                    B_success = true
                    if rho < rho0
                        Delta = min(beta_succ*Delta, Delta0)
                    else
                        Delta = Delta
                    end
                    if satisfies_state_inequality_constraints(SCPproblem, model, X_sol, U_sol, Xp, Up, Delta)
                        omega = omega0
                    else
                        omega = gamma_fail * omega
                    end
                end
            else
                Delta = Delta
                omega     = gamma_fail * omega
                B_success = false
            end
    #         if convergence_metric(model,X,U,Xp,Up) <0.6
    #             println("Convergence metric very small. Decreasing Delta.")
    #             Delta = Delta0./(2^it)
    #         end

        else # always accept first solution
            X = copy(X_sol)
            U = copy(U_sol)
        end


        # -----------

        push!(X_all, copy(X))
        push!(U_all, copy(U))


        it += 1

        diff_with_prev = norm(copy(X) - copy(Xp), Inf)
    end
    if (B_success && convergence_metric(model,X,U,Xp,Up)<convergence_threshold)
        return true
    else
        return false
    end
end

solve_gusto (generic function with 1 method)

In [5]:
#Start setup time
N = 50
MAX_ITERATIONS_NB = 100

model = Astrobee()

# Pick the init and goal positions
# Small corner maneuver
r_init = [11.2; -0.8; 5.6]
r_goal = [10.9; 3.0; 5.0]
q_init = [1.; 0.; 0.; 0.]
q_goal = [1.; 0.5; 0.5; 0.5]

model.x_init = [r_init; zeros(3); q_init; zeros(3)]
model.x_final = [r_goal; zeros(3); q_goal; zeros(3)]
model.tf_guess = 10.

Xp,Up = initialize_trajectory(model, N)
X, U  = copy(Xp), copy(Up)

# X, U  = copy(Xp) .+ 2., copy(Up) .+ 2.

SCPproblem = GuSTOProblem(model, N, Xp, Up)

x_dim, u_dim = model.x_dim, model.u_dim

Delta0 = 1000.
omega0 = 1.
omegamax = 1.0e10
epsilon = 1.0e-1
rho0 = 0.01
rho1 = 100
beta_succ = 2.
beta_fail = 0.5
gamma_fail = 5.
convergence_threshold = 1e-4

# Delta0, omega0, omegamax, 
#     epsilon, rho0, rho1, 
#     beta_succ, beta_fail, gamma_fail, 
#     convergence_threshold = get_initial_gusto_parameters(model)

Delta = Delta0
omega = omega0

cvgd = solve_gusto(MAX_ITERATIONS_NB, X, U, Xp, Up, model, Delta0, omega0, omegamax, 
    epsilon, rho0, rho1, 
    beta_succ, beta_fail, gamma_fail, 
    convergence_threshold)
println("Converged: $cvgd")

Converged: false
